# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

Investigate the effects of oxycodone withdrawal during absence / presence of chronic pain with additional treatment by HDAC1/2 inhibition by RBC1HI. Exploration of novel treatments for patients with opioid dependence to lessen withdrawal symptoms and aid transition to non-opioid drugs.

What do the conditions mean?

oxy: oxycodone administered


sal: saline administered in place of oxycodone

What do the genotypes mean?

SNI:spared nerve injury, mice were given surgery to induce chronic pain


Sham: no chronic pain but sham surgery was performed to normalize for surgical stress

**Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.**

What would you do? organize files by treatment conditions in the four possible combinations given in the matrix below:

|          | Oxy        | Sal        |
|----------|------------|------------|
| **Sham** | Sham - Oxy | Sham - Sal |
| **SNI**  | SNI - Oxy  | SNI - Sal  |

**Which groups would you compare to each other?**

compare those sharing at least one condition, i.e. horizontally and vertically across the treatment matrix above such that one of two variables is always constant and controlled for

**Please also mention which outcome you would expect to see from each comparison.**

Sham surgery should lead to lessened addiction as no chronic pain is present, saline injected mice should not be addicted as no opioids were present

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [16]:
import os

import pandas as pd
conditions = pd.read_excel('conditions_runs_oxy_project.xlsx', index_col='Run').drop(columns='Patient').notna()
conditions.columns = conditions.columns.str.lower() # make column names uniform

In [17]:

print(f'Oxycodone True count: {conditions['condition: oxy'].sum()}')
print(f'Saline True count: {conditions['condition: sal'].sum()}')

Oxycodone True count: 8
Saline True count: 8


In [21]:
print(f'SNI-Oxy Count: {conditions[conditions['condition: oxy']]['genotype: sni'].sum()}')
print(f'SNI-Saline Count: {conditions[conditions['condition: sal']]['genotype: sni'].sum()}')
print(f'SHAM-Oxy Count: {conditions[conditions['condition: oxy']]['genotype: sham'].sum()}')
print(f'SHAM-Saline Count: {conditions[conditions['condition: sal']]['genotype: sham'].sum()}')

SNI-Oxy Count: 4
SNI-Saline Count: 4
SHAM-Oxy Count: 4
SHAM-Saline Count: 4


pThey were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [22]:
conditions['base_counts'] = pd.read_csv('base_counts.csv', index_col='Run')

In [24]:
print(conditions.sort_values('base_counts', ascending=True).head(2))

             rna-seq  dna-seq  condition: sal  condition: oxy  genotype: sni  \
Run                                                                            
SRR23195516     True    False           False            True           True   
SRR23195511     True    False           False            True          False   

             genotype: sham  base_counts  
Run                                       
SRR23195516           False   6203117700  
SRR23195511            True   6456390900  


In [28]:
conditions.sort_values('base_counts', ascending=True).head(2).index.to_series().to_csv('fetch_runs.csv', index=False, header=False)

In [30]:
!nextflow run nf-core/fetchngs \
   -profile docker \
   --input fetch_runs.csv \
   --outdir fetch_out


 N E X T F L O W   ~  version 25.04.7

Launching `https://github.com/nf-core/fetchngs` [happy_jepsen] DSL2 - revision: 8ec2d934f9 [master]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/fetchngs v1.12.0-g8ec2d93
------------------------------------------------------
Core Nextflow options
  revision       : master
  runName        : happy_jepsen
  containerEngine: docker
  launchDir      : /Users/patrick/Studium_Local/3.Sem/computational-workflows-2025/notebooks/day_02
  workDir        : /Users/patrick/Studium_Local/3.Sem/computational-workflows-2025/notebooks/day_02/work
  projectDir     : /Users

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.

use the tools given in the methods sections with identical parameters and same 2x2 factorial design
read alignment: HISAT2, read counting: HT-Seq, differential analysis: DeSEQ2, p-value cutoff < 0.05 and absolute log 2(fold change) > 0.5

chain the nf-core pipelines rna-seq and differentialabundance together for analysis

In [59]:
# generate samplesheet
import os
sample_sheet = pd.DataFrame()
sample_sheet['sample'] = pd.read_csv('fetch_runs.csv', header=None)[0]
fastq_dir = 'fetch_out/fastq'
fastq_files = os.listdir(fastq_dir)
sample_sheet['fastq_1'] = [
    next((os.path.join(fastq_dir, f) for f in fastq_files if (str(sample) + '_1') in f), None)
    for sample in sample_sheet['sample']
]
sample_sheet['fastq_2'] = [
    next((os.path.join(fastq_dir, f) for f in fastq_files if (str(sample) + '_2') in f), None)
    for sample in sample_sheet['sample']
]
sample_sheet['strandedness'] = 'auto'
sample_sheet.to_csv('sample_sheet.csv', index=False)

In [77]:
!nextflow run nf-core/rnaseq \
  --input sample_sheet.csv \
  --aligner hisat2 \
  --outdir results_rnaseq \
  -profile docker \
  --genome GRCm38 \
 -c nextflow.config


 N E X T F L O W   ~  version 25.04.7

Launching `https://github.com/nf-core/rnaseq` [hungry_lavoisier] DSL2 - revision: 9738a2df42 [master]


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq 3.21.0
------------------------------------------------------
Input/output options
  input              : sample_sheet.csv
  outdir             : results_rnaseq

Reference genome options
  genome             : GRCm38
  fasta              : s3://ngi-igenomes/igenomes//Mus_musculus/Ensembl/GRCm38/Sequence/WholeGenomeFasta/genome.fa
  gtf                : s3://ngi-igenomes/igenomes//Mus_musculus/Ensembl/GRCm38/Annotation/Genes/genes.gtf
  gene_bed           : s3://ngi-igenomes/igenomes//Mus_musculus/Ensembl/GRCm38/Annotation/Genes/genes

does not actually run fully due to limited system resources, tried fixing with config but did not actually work